In [75]:
import json
import nltk
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [76]:
#Python library for interacting with JIRA via REST APIs.
from jira import JIRA
#connect to the jira server using HTTP basic auth
jira = JIRA(basic_auth=('bsse0914@iit.du.ac.bd', 'MbaYDePZRnDgAdL062r83605'), options={'server':'https://pg-req.atlassian.net'})

In [77]:
projects = jira.projects()
num_of_projects=len(projects)
issueCount=0
for i in range(0,num_of_projects):
    print(projects[i])
    #jql
    issues_in_proj = jira.search_issues('project='+str(projects[i]))
    print(len(issues_in_proj))
    issueCount+=len(issues_in_proj)
print(issueCount)

C1R
12
FAZ
5
FEP
4
RMS
0
SAAJ
8
29


In [78]:
import re
import string 
def remove_punctuation(text):
    no_punc = "".join([c for c in text if c not in string.punctuation])
    return no_punc.lower()

def tokenize(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    porter_stemmer= PorterStemmer()
    lemmatizer=WordNetLemmatizer()

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    terms_list = []
    #stemmed_sentence = []
    
    #Stemming is a little more aggressive. It cuts off prefixes and/or endings of words
    #based on common ones. It can sometimes be helpful, but not always because often times 
    #the new word is so much a root that it loses its actual meaning. 
    #Lemmatizing, on the other hand, maps common words into one base. 
    #Unlike stemming though, it always still returns a proper word that can be found in the dictionary.
    for w in word_tokens:
        if w not in stop_words:
            #w=porter_stemmer.stem(w)
            w=lemmatizer.lemmatize(w)
            terms_list.append(w)

    #print(word_tokens)
    #print("terms list")
    #print(terms_list)
    
    return terms_list

In [79]:
cnt=0
assignee_terms_map={}
terms_assignee_map={}
terms_assignee_list=[]
account_id_map={}

for p in projects:
    pro_key='project='
    pro_key+=str(p.key)
    project_key=pro_key
    size = 100
    initial = 0
    issueCount=0
    while True:
        start= initial*size
        issues = jira.search_issues(project_key,  start,size)
        issueCount+=len(issues)
        if len(issues) == 0:
            break
        initial += 1
        for issue in issues:
            if issue.fields.assignee and issue.fields.assignee!=None:
                cnt+=1
                summary = issue.fields.summary
                des = issue.fields.description
                assignee = str(issue.fields.assignee)
                account_id = str(issue.fields.assignee.accountId)
                #email = str(issue.fields.assignee.emailAddress)
                creation_date = str(issue.fields.created)
            
                assignee_date = {}
                assignee_date['assignee']=assignee
                assignee_date['date']=creation_date
                
                des=remove_punctuation(des)
 
                bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
                des = ''.join(i for i in des if not i in bad_chars)
                tokenized_des=tokenize(des)
        
                summary=remove_punctuation(summary)
 
                bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
                summary = ''.join(i for i in summary if not i in bad_chars)
                tokenized_summary=tokenize(summary)
               
                #list of all terms existing in an issue
                dupli_terms=tokenized_des+tokenized_summary
            
                terms = list(dict.fromkeys(dupli_terms))
                
                keylist=[]
                for key,val in assignee_terms_map.items():
                    keylist.append(key)
                                
                if assignee in keylist:
                    assignee_terms_map[assignee]=assignee_terms_map[assignee]+terms
                else:
                    assignee_terms_map[assignee]=terms
                    account_id_map[assignee]=account_id
                    #account_id_map[assignee]['email']=email
    
                #terms_assignee_list.append(assignee)
                terms_assignee_list.append(assignee_date)
                
                for term in terms:
                    if term in terms_assignee_map:
                        terms_assignee_map[term]=terms_assignee_map[term]+terms_assignee_list
                    else:
                        terms_assignee_map[term]=terms_assignee_list  
                                        
                terms_assignee_list=[]

#print("Final assignee terms map:")
#print(assignee_terms_map)

print(len(terms_assignee_map))
print("Final terms assignee map:")
print(terms_assignee_map)

print("Account ID map:")
print(account_id_map)

137
Final terms assignee map:
{'write': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:57:26.760-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:56:19.414-0700'}, {'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'icse': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}], 'paper': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:57:26.760-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}, {'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'including': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}], 'finding': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08

In [80]:
db_assignee_list = []
for key in account_id_map:
    temp_dict={}
    temp_dict['name'] = key
    temp_dict['assignee_id'] = account_id_map[key]
    db_assignee_list.append(temp_dict)

print(db_assignee_list)

mycol = mydb["Assignee"]
x = mycol.insert_many(db_assignee_list)

[{'name': 'Moumita Asad', 'assignee_id': '5f5764e4bea5be0068101b9a'}, {'name': 'Noshin Tahsin', 'assignee_id': '5e299526bf04010e70c42927'}, {'name': 'Mridha Md. Nafis Fuad', 'assignee_id': '5ae1622f027e7a2ebfcc12d1'}, {'name': 'Toukir Ahammed', 'assignee_id': '5ca99ba2e929a772795e4ac9'}, {'name': 'A. T. M. Fazlay Rabbi', 'assignee_id': '5e9412627bc0680c2ca4dd39'}]


In [81]:
from pprint import pprint
pprint(terms_assignee_map)

{'404': [{'assignee': 'A. T. M. Fazlay Rabbi',
          'date': '2020-09-06T09:32:06.928-0700'}],
 'accordingly': [{'assignee': 'Toukir Ahammed',
                  'date': '2020-09-08T03:46:00.806-0700'}],
 'ace': [{'assignee': 'Noshin Tahsin', 'date': '2020-09-05T02:55:07.811-0700'}],
 'add': [{'assignee': 'Noshin Tahsin', 'date': '2021-02-20T00:14:00.264-0800'},
         {'assignee': 'Noshin Tahsin', 'date': '2020-09-06T09:29:11.791-0700'}],
 'addon': [{'assignee': 'Noshin Tahsin',
            'date': '2020-09-05T02:55:07.811-0700'}],
 'along': [{'assignee': 'Noshin Tahsin',
            'date': '2020-09-06T09:29:11.791-0700'}],
 'analysis': [{'assignee': 'Moumita Asad',
               'date': '2020-09-08T03:58:33.865-0700'},
              {'assignee': 'Noshin Tahsin',
               'date': '2020-09-08T03:58:02.767-0700'}],
 'arxiv': [{'assignee': 'Noshin Tahsin',
            'date': '2020-09-08T03:57:26.760-0700'}],
 'assignee': [{'assignee': 'Noshin Tahsin',
               'date':

In [82]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
print("now =", now)
currentTime = now.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
print(currentTime)

now = 2021-04-01 06:20:37.994186
2021-04-01T06:20:37.994186


In [83]:
import copy

db_term_assignee_list = []
for key in terms_assignee_map:
    temp_dict={}
    temp_dict['term'] = key
    val_list = []
    val_list = copy.deepcopy(terms_assignee_map[key])
    temp_dict['developer_expertise'] = val_list
    temp_dict['last_updated'] = currentTime
    db_term_assignee_list.append(temp_dict)

print(db_term_assignee_list)

[{'term': 'write', 'developer_expertise': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:57:26.760-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:56:19.414-0700'}, {'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'last_updated': '2021-04-01T06:20:37.994186'}, {'term': 'icse', 'developer_expertise': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}], 'last_updated': '2021-04-01T06:20:37.994186'}, {'term': 'paper', 'developer_expertise': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:57:26.760-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}, {'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'last_updated': '2021-04-01T06:20:37.994186'}, {'term': 

In [84]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["JiraPlugin"]
print(myclient.list_database_names())

['JiraPlugin', 'admin', 'config', 'local']


In [85]:
mycol = mydb["TermAssigneeMap"]
x = mycol.insert_many(db_term_assignee_list)
#print list of the _id values of the inserted documents:
#print(x.inserted_ids)

In [86]:
myquery = { "term": "backend" }
mydoc = mycol.find(myquery)
for x in mydoc:
    print(x)

{'_id': ObjectId('6065c2527c14cf70f6546ad8'), 'term': 'backend', 'developer_expertise': [{'assignee': 'A. T. M. Fazlay Rabbi', 'date': '2020-12-25T07:29:57.045-0800'}, {'assignee': 'A. T. M. Fazlay Rabbi', 'date': '2020-09-06T09:32:06.928-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-06T09:29:51.314-0700'}], 'last_updated': datetime.datetime(2021, 4, 1, 5, 48, 4, 567000)}
{'_id': ObjectId('6065c7bb7c14cf70f6546b6e'), 'term': 'backend', 'developer_expertise': [{'assignee': 'A. T. M. Fazlay Rabbi', 'date': '2020-12-25T07:29:57.045-0800'}, {'assignee': 'A. T. M. Fazlay Rabbi', 'date': '2020-09-06T09:32:06.928-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-06T09:29:51.314-0700'}], 'last_updated': '2021-04-01T06:16:21.567200'}
{'_id': ObjectId('6065c8a67c14cf70f6546c03'), 'term': 'backend', 'developer_expertise': [{'assignee': 'A. T. M. Fazlay Rabbi', 'date': '2020-12-25T07:29:57.045-0800'}, {'assignee': 'A. T. M. Fazlay Rabbi', 'date': '2020-09-06T09:32:06.928-0700'}, {'as

In [87]:
print(len(assignee_terms_map))
available_devlist=[]

for key,val in assignee_terms_map.items():
    available_devlist.append(key)
print(available_devlist) 
#del assignee_terms_map["None"]
for key,val in  assignee_terms_map.items():
    print(key, "=>", val)
    #print(key)
    #print(len(val))   

5
['Moumita Asad', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Toukir Ahammed', 'A. T. M. Fazlay Rabbi']
Moumita Asad => ['write', 'icse', 'paper', 'including', 'finding', 'regression', 'analysis', 'primary', 'demonstration', 'write', 'blog', 'productivity', 'employee', 'wfh', 'publish', 'dsse', 'website', 'explore', 'literature', 'find', 'paper', 'related', 'employee', 'productivity', 'work', 'home']
Noshin Tahsin => ['regression', 'analysis', 'data', 'write', 'paper', 'finding', 'far', 'arxiv', 'create', 'pie', 'chart', 'data', 'visualization', 'clean', 'preprocess', 'data', 'change', 'column', 'name', 'drop', 'invalid', 'encode', 'categorical', 'find', 'productivityrelated', 'factor', 'related', 'paper', 'prepare', 'survey', 'questionnare', 'getting', 'list', 'view', 'queried', 'result', 'user', 'vscode', 'extension', 'exploring', 'feature', 'better', 'strategy', 'viewing', 'implementing', 'search', 'viewer', 'error', 'need', 'solve', 'add', 'new', 'table', 'ui', 'deploy', 'backend',

In [88]:
db_assignee_terms_list = []
for key in assignee_terms_map:
    temp_dict={}
    temp_dict['assignee'] = key
    val_list = []
    val_list = copy.deepcopy(assignee_terms_map[key])
    temp_dict['term_info'] = val_list
    db_assignee_terms_list.append(temp_dict)

print(db_assignee_terms_list)

mycol = mydb["AssigneeTermsMap"]
x = mycol.insert_many(db_assignee_terms_list)

[{'assignee': 'Moumita Asad', 'term_info': ['write', 'icse', 'paper', 'including', 'finding', 'regression', 'analysis', 'primary', 'demonstration', 'write', 'blog', 'productivity', 'employee', 'wfh', 'publish', 'dsse', 'website', 'explore', 'literature', 'find', 'paper', 'related', 'employee', 'productivity', 'work', 'home']}, {'assignee': 'Noshin Tahsin', 'term_info': ['regression', 'analysis', 'data', 'write', 'paper', 'finding', 'far', 'arxiv', 'create', 'pie', 'chart', 'data', 'visualization', 'clean', 'preprocess', 'data', 'change', 'column', 'name', 'drop', 'invalid', 'encode', 'categorical', 'find', 'productivityrelated', 'factor', 'related', 'paper', 'prepare', 'survey', 'questionnare', 'getting', 'list', 'view', 'queried', 'result', 'user', 'vscode', 'extension', 'exploring', 'feature', 'better', 'strategy', 'viewing', 'implementing', 'search', 'viewer', 'error', 'need', 'solve', 'add', 'new', 'table', 'ui', 'deploy', 'backend', 'heroku', 'assignee', 'suggestion', 'table', 'ad

In [89]:
#tokenizing the new issue
#new_des=data["projects"][0]['issues'][14]['description']
#all_new_issues=jira.search_issues('project='+str(projects[0]))
#new_issue=all_new_issues[0]
new_issue_key = 'C1R-12'
new_issue=jira.issue(new_issue_key)
new_issue_creation_date = new_issue.fields.created
#issue=jira.issue('SAAJ-4')
#print("issue status: ",issue.fields.status.name)
creation_date = new_issue.fields.created
current_time = creation_date
#print("creation date: ",creation_date)
#resolution_date=issue.fields.resolutiondate
#print("resolution date:",resolution_date)
#print("date")
#print(new_issue_creation_date)
new_des=new_issue.fields.description

new_des=remove_punctuation(new_des)
bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
new_des = ''.join(i for i in new_des if not i in bad_chars)
tokenized_new_des=tokenize(new_des)

new_summary=new_issue.fields.summary

new_summary=remove_punctuation(new_summary)
bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
new_summary = ''.join(i for i in new_summary if not i in bad_chars)
tokenized_new_summary=tokenize(new_summary)

new_terms=tokenized_new_des+tokenized_new_summary
print(new_terms)

#now I have the new issue terms

['explore', 'literature', 'find', 'tool', 'paper', 'icse', 'related', 'regression', 'analysis', 'related', 'data', 'find', 'tool', 'paper', 'icse']


In [90]:
#now calculate score for developer suggestion
#print(len(new_terms))
newterm_dev_map={}
#def getDevTermUsageInfo():
cnt=0
for newterm in new_terms:
    if newterm in terms_assignee_map:
        cnt=cnt+1
        #print(newterm)
        #print(terms_assignee_map[newterm])
        newterm_dev_map[newterm]=terms_assignee_map[newterm]
        #returns a devlist for a single newterm
        #developers who has previously worked on that term
        #one dev name can occur multiple times
        #which implies he has worked on that term multiple times-more experienced!

#print(newterm_dev_map['everything'])
print(newterm_dev_map)

{'explore': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}], 'literature': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}], 'find': [{'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}], 'tool': [{'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'paper': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:57:26.760-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}, {'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'icse': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}], 'related': [{'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:

In [91]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
#print("now =", now)
currentTime = now.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
print("current date and time =", current_time)	

current date and time = 2020-09-08T04:32:18.147-0700


In [92]:
print(newterm_dev_map)

{'explore': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}], 'literature': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}], 'find': [{'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}], 'tool': [{'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'paper': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:57:26.760-0700'}, {'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:59.028-0700'}, {'assignee': 'Mridha Md. Nafis Fuad', 'date': '2020-09-05T03:57:46.757-0700'}], 'icse': [{'assignee': 'Moumita Asad', 'date': '2020-09-08T03:58:33.865-0700'}], 'related': [{'assignee': 'Noshin Tahsin', 'date': '2020-09-08T03:44:53.680-0700'}, {'assignee': 'Moumita Asad', 'date': '2020-09-08T03:42:

In [93]:
dev_termInfo={}
newterm_dev_list =[]
newterm_dev_date = []

for dev in available_devlist:
    #print(dev)
    term_info={}
    #check if he has used a newterm and get the newterm
    for key in newterm_dev_map:
        new_time_diff=798798090
        newterm_dev_list =[]
        term_list_len = len(newterm_dev_map[key])
        for i in range (0,term_list_len):
            #print(newterm_dev_map[nterm][i])
            newterm_dev_list.append(newterm_dev_map[key][i]['assignee'])
            newterm_dev_date.append(newterm_dev_map[key][i]['date'])
            
        if dev in newterm_dev_list:
            #now i need the key-key is the newterm
            #calculate term info for the key
            #just need the fixfrequency
            #how many times dev is in newterm_dev_map[key]
            devnames=newterm_dev_list       
            #print(devnames)
            dev_count=devnames.count(dev)
            #print(key)
            #print(dev_count)
            #List-devcount,termusedate
            for d in newterm_dev_date:
                datetrack=d
                #ekhn er date thke shb thke kache jeta first e init inf diye
                current_time_converted = datetime.strptime(current_time,'%Y-%m-%dT%H:%M:%S.%f%z')
                datetrack_converted = datetime.strptime(datetrack,'%Y-%m-%dT%H:%M:%S.%f%z')
            
                time_diff = current_time_converted - datetrack_converted
                if(time_diff.days<new_time_diff):
                    kept_datetrack = datetrack 
                
            devcount_lastuse = {}
            devcount_lastuse['devcount']=dev_count
            devcount_lastuse['lastuse']=kept_datetrack
            
            term_info[key]=devcount_lastuse #add term_use_date : last fixing time of a term by a developer
            #print(term_info)
            
    dev_termInfo[dev]=term_info

    print(dev_termInfo)

{'Moumita Asad': {'explore': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'literature': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'find': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'paper': {'devcount': 2, 'lastuse': '2020-09-05T03:57:46.757-0700'}, 'icse': {'devcount': 1, 'lastuse': '2020-09-08T03:58:33.865-0700'}, 'related': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'regression': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'analysis': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}}}
{'Moumita Asad': {'explore': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'literature': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'find': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'paper': {'devcount': 2, 'lastuse': '2020-09-05T03:57:46.757-0700'}, 'icse': {'devcount': 1, 'lastuse': '2020-09-08T03:58:33.865-0700'}, 'related': {'devcount': 1, 'lastuse': '2020-09

In [94]:
for key,value in dev_termInfo.items():
    print(key," => ",value)
#Moumita Asad  =>  {'data': 2, (date last worked)}
#now i have the complex map of dev term term-info

Moumita Asad  =>  {'explore': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'literature': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'find': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'paper': {'devcount': 2, 'lastuse': '2020-09-05T03:57:46.757-0700'}, 'icse': {'devcount': 1, 'lastuse': '2020-09-08T03:58:33.865-0700'}, 'related': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'regression': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'analysis': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}}
Noshin Tahsin  =>  {'find': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'paper': {'devcount': 2, 'lastuse': '2020-09-05T03:57:46.757-0700'}, 'related': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'regression': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'analysis': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'data': {'devcount': 3, 'lastuse': '2020

In [95]:
import time
from datetime import datetime, timedelta

prediction_list=[]

def devFixFreq(term):
    dev_use_count=0
    for dev in assignee_terms_map:
        terms_used=assignee_terms_map[dev]
        if term in terms_used:#if term in terms used keylist
            dev_use_count=dev_use_count+1
    return dev_use_count
    
dev_score={} 
for key in dev_termInfo:
    expertise=0
    termInfo=dev_termInfo[key]
    print(termInfo)
    for each_term in termInfo:
        term_freq=termInfo[each_term]['devcount']
        term_using_date_latest = termInfo[each_term]['lastuse']
        #print(info_term)
        #print(term_freq)
        ##dev=number of devs in the project 
        #len(available_devlist)
        
        #devfixfreq() : returns numof devs who fixed the term related bugs
        num_of_devs=len(available_devlist)
        tfIdf=term_freq*math.log(num_of_devs/devFixFreq(each_term))
        #difff = new issue creation - term using date
       
        c_new_issue_creation_date = datetime.strptime(new_issue_creation_date,'%Y-%m-%dT%H:%M:%S.%f%z')
        c_term_using_date_latest = datetime.strptime(term_using_date_latest,'%Y-%m-%dT%H:%M:%S.%f%z')

        difff = c_new_issue_creation_date - c_term_using_date_latest
        #print(difff.days)
        if difff.days!=0:
            fix_date=(1/devFixFreq(each_term))+(1/(difff.days**(1/2)))
        else:
            difff = 14 #one-sprint
            fix_date=(1/devFixFreq(each_term))+(1/(difff**(1/2)))
        expertise+=tfIdf*fix_date
    #end of for
    dev_score[key]=expertise
#end of for
print("\nSCORE: ")
print(dev_score)
    

{'explore': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'literature': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'find': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'paper': {'devcount': 2, 'lastuse': '2020-09-05T03:57:46.757-0700'}, 'icse': {'devcount': 1, 'lastuse': '2020-09-08T03:58:33.865-0700'}, 'related': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'regression': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'analysis': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}}
{'find': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'paper': {'devcount': 2, 'lastuse': '2020-09-05T03:57:46.757-0700'}, 'related': {'devcount': 1, 'lastuse': '2020-09-08T03:42:59.028-0700'}, 'regression': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'analysis': {'devcount': 1, 'lastuse': '2020-09-08T03:58:02.767-0700'}, 'data': {'devcount': 3, 'lastuse': '2020-09-08T03:53:11.871-0700'}}
{'tool': 

In [100]:
sorted_dev_score = sorted(dev_score.items() , reverse=True, key=lambda x: x[1])
# Iterate over the sorted sequence
scoreList = []
for elem in sorted_dev_score :
    print(elem[0] , " ::" , elem[1] )
    scoreList.append(elem[1])

Moumita Asad  :: 9.861273361263468
Noshin Tahsin  :: 4.662935793778088
Mridha Md. Nafis Fuad  :: 3.3106469422645524
Toukir Ahammed  :: 0.30679909853038445
A. T. M. Fazlay Rabbi  :: 0


In [101]:
a=[]
a_id=[]
for i in range (0,len(sorted_dev_score)):
    a.append(sorted_dev_score[i][0])
    a_id.append(account_id_map[sorted_dev_score[i][0]])
    print(sorted_dev_score[i][0])
    print("Account ID")
    print(account_id_map[sorted_dev_score[i][0]])
    print("\n")

print(a)
print(a_id)

Moumita Asad
Account ID
5f5764e4bea5be0068101b9a


Noshin Tahsin
Account ID
5e299526bf04010e70c42927


Mridha Md. Nafis Fuad
Account ID
5ae1622f027e7a2ebfcc12d1


Toukir Ahammed
Account ID
5ca99ba2e929a772795e4ac9


A. T. M. Fazlay Rabbi
Account ID
5e9412627bc0680c2ca4dd39


['Moumita Asad', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Toukir Ahammed', 'A. T. M. Fazlay Rabbi']
['5f5764e4bea5be0068101b9a', '5e299526bf04010e70c42927', '5ae1622f027e7a2ebfcc12d1', '5ca99ba2e929a772795e4ac9', '5e9412627bc0680c2ca4dd39']


In [104]:
db_suggestion_list = []
temp_dict={}
#project_id: ‘RP’
#issue_id: ‘RP-1’
project_id = new_issue_key.split('-')
temp_dict['project_id'] = project_id[0]
temp_dict['issue_id'] = new_issue_key
temp_dict['suggestion_list_id'] = a_id
temp_dict['suggestion_list_name'] = a
temp_dict['score_list'] = scoreList
temp_dict['assignee_id'] = a_id[0]
temp_dict['assignee_name'] = a[0]
temp_dict['assigning_date'] = currentTime
db_suggestion_list.append(temp_dict)

print(db_suggestion_list)

mycol = mydb["Suggestions"]
x = mycol.insert_many(db_suggestion_list)

[{'project_id': 'C1R', 'issue_id': 'C1R-12', 'suggestion_list_id': ['5f5764e4bea5be0068101b9a', '5e299526bf04010e70c42927', '5ae1622f027e7a2ebfcc12d1', '5ca99ba2e929a772795e4ac9', '5e9412627bc0680c2ca4dd39'], 'suggestion_list_name': ['Moumita Asad', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Toukir Ahammed', 'A. T. M. Fazlay Rabbi'], 'score_list': [9.861273361263468, 4.662935793778088, 3.3106469422645524, 0.30679909853038445, 0], 'assignee_id': '5f5764e4bea5be0068101b9a', 'assignee_name': 'Moumita Asad', 'assigning_date': '2021-04-01T06:20:38.521479'}]
